In [2]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
import pyarrow as pa
import fastparquet as fp
from textblob import TextBlob
from fastapi.responses import HTMLResponse

In [3]:
df_reviews = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\user_reviews_clean.parquet")
df_games = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\user_games_clean.parquet")
df_items = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\user_items_clean.parquet")
df_recomendacion = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_recomendacion.parquet")
df_mix = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_mix.parquet")
df_conteo = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_conteo.parquet")
df_posicion = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_posicion.parquet")
df_ugenre = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_ugenre.parquet")
df_funsentiment = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_funsentiment.parquet")
df_ITGA = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_ITGA.parquet")
df_countreviews = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_countreviews.parquet")
df_sentiment = pd.read_parquet(r"C:\Users\ACER\Documents\HENRY\LABS\PI01\JUPYTER NOTEBOOKS\data\df_sentiment.parquet")


In [16]:
def userdata(user_id: str):
    '''
    <b>Objetivo:</b>
    
    Devuelve, para el usuario pasado como parámetro,
    el dinero gastado, el porcentaje de recomendación 
    y la cantidad de items del mismo.
    
    <b>Argumento:</b>
    
    user_id (str): ID de identifiación del usuario 
    
    <b>Ejemplo:</b>
    
    user_id: 76561197970982479
    
    '''
    # Filtrar los DataFrames por el usuario de interés
    user_df = df_ITGA[df_ITGA['user_id'] == user_id]
    user_df_2 = df_reviews[df_reviews['user_id'] == user_id]
    user_df_3 = df_items[df_items['user_id'] == user_id]

    # Calcular la suma de la columna 'price' (dinero gastado)
    money = round(user_df['price'].sum(), 2)
    
    # Calcular la cantidad total de recomendaciones
    tot_recommend = user_df_2['recommend'].sum()
    
    # Obtener la cantidad de items del usuario
    tot_items = user_df_3['items_count'].iloc[0].item()

    return {
        'Usuario': user_id,
        'Cantidad de dinero gastado': money,
        'Porcentaje de recomendación': round((tot_recommend / tot_items) * 100, 2),
        'Cantidad de items': tot_items
    }
    


@app.get("/countreviews/{inicio}/{fin}", name="COUNTREVIEWS")

In [17]:

def countreviews(inicio: str, fin: str):
    '''
    <b>Objetivo:</b>
    
    Devuelve la cantidad de usuarios que realizaron reviews entre 
    las fechas dadas y el porcentaje de recomendacion de esos usuarios.
    
    <b>Argumentos:</b>
    
    inicio (str): Fecha de inicio del período a evaluar.
    
    fin (str): Fecha de fin del período a evaluar
    
    <b>Ejemplo:</b>
    
    inicio: 2010-12-30
    
    fin: 2013-06-25
    '''
  
    inicio = pd.to_datetime(inicio)
    fin = pd.to_datetime(fin) 
    
    # Filtra las filas del DataFrame que estén dentro del rango de fechas
    reviews_fechas = df_reviews[(df_reviews['posted'] >= inicio) & (df_reviews['posted'] <= fin)]
    
    # Calcula la cantidad de usuarios únicos que realizaron reviews en ese período
    cantidad_usuarios = reviews_fechas['user_id'].nunique()
    
    # Calcula el porcentaje de recomendación
    porcentaje_recomendacion = round(
        (reviews_fechas['recommend'].sum() / reviews_fechas.shape[0]) * 100, 2)
    
    return {
        'Cantidad de usuarios': cantidad_usuarios,
        'Porcentaje de recomendación': porcentaje_recomendacion
    }


In [18]:
def userforgenre(genre):
    '''
    <b>Objetivo:</b>
    
    Devuelve el TOP 5 de usuarios con más horas jugadas en un género específico.

    <b>Argumento:</b>
    
    genre (str): El género de juegos para el que se desea obtener el TOP 5 de usuarios.
    
    <b>Ejemplo:</b>
    
    genero: Adventure
    
    '''

    # Filtrar el DataFrame para obtener datos específicos del género y ordenar por horas jugadas
    top_users = df_ugenre[df_ugenre['genres'] == genre] \
        .sort_values(by='playtime_forever', ascending=False) \
        .head(5) \
        .reset_index(drop=True)

    # Crear una lista de diccionarios para representar el resultado
    top_users_list = [
        {
            'user_id': row['user_id'],
            'user_url': row['user_url'],
            'playtime_forever': row['playtime_forever']
        }
        for index, row in top_users.iterrows()
    ]

    return  {
        'Genero': genre, 
        'TOP 5 de usuarios': top_users_list
    }



In [19]:
def developer(desarrollador):
    '''
    <b>Objetivo:</b>
    
    Devuelve cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

    <b>Argumento:</b>
    
    desarrolador (str): El developer del juego (item) para el cual se desea obtener los valores mencionados. 
    
    <b>Ejemplo:</b>
    
    Desarrollador: Valve

    '''

    # Filtramos el DataFrame por el desarrollador dado
    developer_df = df_ITGA[df_ITGA['developer'] == desarrollador]

   


In [20]:

def sentiment_analysis(anio: int):
    '''
    <b>Objetivo:</b>
    
    Devuelve un DataFrame con la cantidad de registros de reseñas de usuarios categorizados por análisis de sentimiento para un año específico.
    
    <b>Argumentos:</b>
    
    anio (int): Año en el cual queremos obtener el análisis de sentimiento.
    
    <b>Ejemplo:</b>
    
    anio: 2012
   
    '''
    # Filtramos el DataFrame según el año definido como argumento
    df_filtered = df_funsentiment[df_funsentiment['anio'] == anio]

    # Obtenemos la cantidad de valores positivos, negativos y neutros
    sentiment_counts = df_filtered['sentimiento'].value_counts().to_dict()

    return sentiment_counts


In [22]:
def genero_posicion(genre):
    '''
    Esta función devuelve la posición en el ranking de horas jugadas para un género dado.

    Argumento: 
    genre (str): El género de juegos del cual se quiere conocer el ranking de horas jugadas. 

    Retorna:
    dict: Un diccionario con el género y su posición en el ranking.
    '''
    
    # Filtrar el ranking según el género
    genre_rank = df_posicion[df_posicion['genero'] == genre]

    if not genre_rank.empty:
        position = genre_rank['Posicion'].iloc[0]
        return {'El género': genre, 'se encuentra en la posición': position}
    else:
        return {'El género': genre, 'mensaje': 'No se encontró en el ranking'}

# Ejemplo de uso:
genre_info = genero_posicion('Sports')
print(genre_info)

KeyError: 'genero'